Demonstrating how to get DonkeyCar Tub files into a PyTorch/fastai DataBlock

In [ ]:
from fastai.data.all import *
from fastai.vision.all import *
from fastai.data.transforms import ColReader, Normalize, RandomSplitter
from fastai.metrics import rmse
import torch
from torch import nn
from torch.nn import functional as F

In [ ]:
from donkeycar.parts.tub_v2 import Tub
import pandas as pd
from pathlib import Path

In [ ]:
from malpi.dk.train import preprocessFileList, get_data, get_learner, get_autoencoder, train_autoencoder
from malpi.dk.vae import VanillaVAE
from malpi.dk.train import get_dataframe

In [ ]:
def learn_resnet():
    learn2 = cnn_learner(dls, resnet18, loss_func=MSELossFlat(), metrics=[rmse], cbs=ActivationStats(with_hist=True))
    learn2.fine_tune(5)
    
    learn2.recorder.plot_loss()
    learn2.show_results(figsize=(20,10))

The below code is modified from: https://github.com/cmasenas/fastai_navigation_training/blob/master/fastai_train.ipynb.

TODO: Figure out how to have multiple output heads

In [ ]:
def test_one_transform(name, inputs, df_all, batch_tfms, item_tfms, epochs, lr):
    dls = get_data(inputs, df_all=df_all, batch_tfms=batch_tfms, item_tfms=item_tfms)
    callbacks = [CSVLogger(f"Transform_{name}.csv", append=True)]
    learn = get_learner(dls)
    #learn.no_logging() #Try this to block logging when doing many training test runs
    learn.fit_one_cycle(epochs, lr, cbs=callbacks)
    #learn.recorder.plot_loss()
    #learn.show_results(figsize=(20,10))

In [ ]:
# Train multipel times using a list of Transforms, one at a time.
# Compare mean/stdev of best validation loss (or rmse?) for each Transform
df_all = get_dataframe("track1_warehouse.txt")
transforms = [None]
transforms.extend( [*aug_transforms(do_flip=False, size=128)] )
for tfm in transforms:
    name = "None" if tfm is None else str(tfm.__class__.__name__)
    print( f"Transform: {name}" )
    for i in range(5):
        print( f"   Run {i+1}" )
        test_one_transform(name, "track1_warehouse.txt", df_all, None, 5, 3e-3)

In [ ]:
def visualize_learner( learn ):
    #dls=nav.dataloaders(df, bs=512)
    preds, tgt = learn.get_preds(dl=[dls.one_batch()])

    plt.title("Target vs Predicted Steering", fontsize=18, y=1.0)
    plt.xlabel("Target", fontsize=14, labelpad=15)
    plt.ylabel("Predicted", fontsize=14, labelpad=15)
    plt.plot(tgt.T[0], preds.T[0],'bo')
    plt.plot([-1,1],[-1,1],'r', linewidth = 4)
    plt.show()

    plt.title("Target vs Predicted Throttle", fontsize=18, y=1.02)
    plt.xlabel("Target", fontsize=14, labelpad=15)
    plt.ylabel("Predicted", fontsize=14, labelpad=15)
    plt.plot(tgt.T[1], preds.T[1],'bo')
    plt.plot([0,1],[0,1],'r', linewidth = 4)
    plt.show()

In [ ]:
learn.export()

In [ ]:
df_all = get_dataframe("track1_warehouse.txt")
dls = get_data("track1_warehouse.txt", df_all=df_all, batch_tfms=None)

In [ ]:
len(dls.valid_ds)

In [ ]:
learn = get_learner(dls)
learn.fit_one_cycle(15, 3e-3)

In [ ]:
visualize_learner(learn)

In [ ]:
learn.export('models/track1_v2.pkl')

In [ ]:
def clear_pyplot_memory():
    plt.clf()
    plt.cla()
    plt.close()

df_all = get_dataframe("track1_warehouse.txt")

transforms=[None,
            RandomResizedCrop(128,p=1.0,min_scale=0.5,ratio=(0.9,1.1)),
            RandomErasing(sh=0.2, max_count=6,p=1.0),
            Brightness(max_lighting=0.4, p=1.0),
            Contrast(max_lighting=0.4, p=1.0),
            Saturation(max_lighting=0.4, p=1.0)]
#dls = get_data(None, df_all, item_tfms=item_tfms, batch_tfms=batch_tfms)

for tfm in transforms:
    name = "None" if tfm is None else str(tfm.__class__.__name__)
    if name == "RandomResizedCrop":
        item_tfms = tfm
        batch_tfms = None
    else:
        item_tfms = None
        batch_tfms = tfm
        
    dls = get_data("track1_warehouse.txt",
                   df_all=df_all,
                   item_tfms=item_tfms, batch_tfms=batch_tfms)

    dls.show_batch(unique=True, show=True)
    plt.savefig( f'Transform_{name}.png' )
#clear_pyplot_memory()

In [ ]:
learn, dls = train_autoencoder( "tracks_all.txt", 5, 3e-3, name="ae_test1", verbose=False )

In [ ]:
learn.recorder.plot_loss()
learn.show_results(figsize=(20,10))
#plt.savefig(name + '.png')

In [ ]:
idx = 0

In [ ]:
idx += 1
im1 = dls.one_batch()[0]
im1_out = learn.model.forward(im1)
show_image(im1[idx])
show_image(im1_out[idx])

In [ ]:
input_file="tracks_all.txt"
item_tfms = [Resize(64,method="squish")]
dls_1 = get_data(input_file, item_tfms=item_tfms, verbose=True, autoencoder=True)

In [ ]:
vae = VanillaVAE(128, 64)
#learn = Learner(dls_1, vae, loss_func=vae.loss_function)

In [ ]:
vae

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(100, 4.7e-4,
                   cbs=[EarlyStoppingCallback(monitor='valid_loss', min_delta=0.0, patience=5)])

In [ ]:
#vae
#learn.recorder.plot_loss()
#learn.show_results(figsize=(20,10))
idx = 2
im1 = dls_1.one_batch()[0]
im1_out, inp, mu, log_var = learn.model.forward(im1)
show_image(im1[idx])
show_image(im1_out[idx])

In [ ]:
learn.export("vae_v2.pkl")

In [ ]:
mu_samp = vae.reparameterize(mu, log_var)
img = vae.decode(mu_samp)
show_image(img[1])

In [ ]:
learn = load_learner("vae_v2.pkl", cpu=False)

In [ ]:
defaults.device = torch.device('cpu')
input_file="tracks_all.txt"
item_tfms = [Resize(128,method="squish")]
dls = get_data(input_file, item_tfms=item_tfms, verbose=False, autoencoder=False)

In [ ]:
name="vae_v3"
input_file="tracks_all.txt"
item_tfms = [Resize(128,method="squish")]
callbacks = [EarlyStoppingCallback(monitor='valid_loss', min_delta=0.0, patience=5)]
epochs = 10
lr = 4.7e-4

vae = VanillaVAE(128, 64)
vae.meta['input'] = input_file
vae.meta['image_size'] = (128,128)
vae.meta['epochs'] = epochs
vae.meta['lr'] = lr

dls = get_data(input_file, item_tfms=item_tfms, verbose=False, autoencoder=True)
vae.meta['train'] = len(dls.train_ds)
vae.meta['valid'] = len(dls.valid_ds)

learn = Learner(dls, vae, loss_func=vae.loss_function)
learn.fit_one_cycle(epochs, lr, cbs=callbacks)
learn.export( name + ".pkl" )

In [ ]:
input_file="tracks_all.txt"
item_tfms = [Resize(128,method="squish")]
df_all = get_dataframe(input_file)
dls = get_data(input_file, df_all=df_all, item_tfms=item_tfms, verbose=False, autoencoder=False)

learn = load_learner("models/vae_v3.pkl", cpu=False)

In [ ]:
mus = []
var = []
outputs = []
total = 0
learn.model.eval()
with torch.no_grad():
    for images, controls in dls.train:
        total += images.shape[0]
        _, _, mu, log_var = learn.forward( images)
        mus.append(mu)
        var.append(log_var)
        outputs.append(controls)
        print( f"Total/len: {total}/{len(outputs)}/{len(mus)}" )
        if total > 200:
            break

mus = torch.stack(mus)
mus = torch.reshape(mus, (mus.shape[0] * mus.shape[1], mus.shape[2])) # combine sub-array and batch dimensions
var = torch.stack(var)
var = torch.reshape(var, (var.shape[0] * var.shape[1], var.shape[2]))
outputs = torch.stack(outputs)


In [ ]:
mus.shape, outputs.shape

In [ ]:
df_256 = df_all[['user/angle','user/throttle']][0:256].copy()
df_256['mu'] = np.array(mus.cpu()).tolist()
df_256['var_log'] = np.array(var.cpu()).tolist()

In [ ]:
z_len = 64
blocks = (RegressionBlock(n_out=z_len), RegressionBlock(n_out=z_len), RegressionBlock(n_out=2))
y_reader = ColReader(['user/angle','user/throttle'])
pascal = DataBlock(blocks=blocks,
                   splitter=RandomSplitter(),
                   get_x=[ColReader("mu"),ColReader("var_log")],
                   get_y=y_reader,
                   item_tfms=None,
                   batch_tfms=None,
                   n_inp=2)

dls = pascal.dataloaders(df_256)


In [ ]:
b1 = dls.one_batch()

In [ ]:
b1[2].shape

In [ ]:
class DK_PreVAE(nn.Module):
    """ A DonkeyCar driver that takes as inputs mu/log_var from a 
        pre-trained VAE. """
    
    def __init__(self, latent_dim, outputs=2):
        self.latent_dim = latent_dim
        self.driver = nn.Sequential(
            torch.nn.Linear(self.latent_dim, 50),
            torch.nn.ReLU(),
            torch.nn.Linear(50, outputs),
            torch.nn.Tanh()
        )
        
    def reparameterize(self, mu: Tensor, logvar: Tensor) -> Tensor:
        """
        Reparameterization trick to sample from N(mu, var) from
        N(0,1).
        :param mu: (Tensor) Mean of the latent Gaussian [B x D]
        :param logvar: (Tensor) Standard deviation of the latent Gaussian [B x D]
        :return: (Tensor) [B x D]
        """
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu

    def forward(self, input: Tensor, **kwargs) -> List[Tensor]:
        # not sure what input will look like. Should be mu and log_var, both of length latent_dim
        z = self.reparameterize(input[0], input[1])
        return self.driver.forward(z)
